In [1]:
import pyrqa
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./data/final.csv')

In [4]:
len(data.session_id.unique())

129

In [5]:
a = data['Primary code '].unique()

In [6]:
a

array(['SMC', 'CM', 'CP', 'SSI', 'SESU', 'SN', 'CE', 'CRF', 'EM', 'TD'],
      dtype=object)

In [7]:
from numpy import asarray
from sklearn.preprocessing import LabelEncoder
# define data
a = data['Primary code ']
# define one hot encoding
encoder = LabelEncoder()
# transform data
encoder.fit(a)

LabelEncoder()

In [8]:
codes_list = data.groupby(['session_id'])['Primary code '].apply(list)

In [23]:
codes_list

session_id
7283_Professor    [SMC, SMC, SESU, SSI, SESU, SSI, SSI, SN, SN, ...
7285_Professor    [SMC, SMC, SMC, SMC, CE, CE, SSI, SESU, SN, CR...
7287_Professor    [SMC, SSI, SSI, SSI, SSI, CRF, CM, SSI, SSI, S...
7288_Professor    [SMC, SESU, SSI, SESU, CRF, CRF, SSI, SSI, SES...
7289_Professor    [SMC, SESU, SESU, SSI, SSI, CM, SN, CRF, SSI, ...
                                        ...                        
7376_apartment    [SMC, SSI, SSI, CRF, SN, CRF, SMC, SSI, SESU, ...
7377_apartment    [SMC, SMC, SSI, SN, SN, CRF, SN, CM, SN, CM, S...
7378_apartment    [SMC, SESU, SSI, CM, SSI, SESU, CE, SSI, SSI, ...
7380_apartment    [SMC, SSI, SSI, SSI, CE, CRF, SSI, SSI, SN, SS...
7388_apartment    [SMC, SSI, SSI, SESU, SSI, SMC, SSI, SSI, CRF,...
Name: Primary code , Length: 129, dtype: object

In [9]:
from pyrqa.analysis_type import Cross
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation

x = []


for i in codes_list:
    data_points_x = encoder.transform(i)
    time_series_x = TimeSeries(data_points_x,
                               embedding_dimension=1,
                               time_delay=0)
    time_series = (time_series_x,
                   time_series_x)
    settings = Settings(time_series,
                        analysis_type=Cross,
                        neighbourhood=FixedRadius(0.5),
                        similarity_measure=EuclideanMetric,
                        theiler_corrector=0)
    computation = RQAComputation.create(settings,
                                        verbose=False)
    result = computation.run()
    result.min_diagonal_line_length = 2
    result.min_vertical_line_length = 2
    result.min_white_vertical_line_length = 2
    #print(result)
    x.append([result.determinism, result.recurrence_rate, result.entropy_diagonal_lines,
             result.longest_diagonal_line, result.laminarity,
             result.divergence, len(data_points_x)])
    

In [10]:
from pyrqa.computation import RPComputation
from pyrqa.image_generator import ImageGenerator
computation = RPComputation.create(settings)
result = computation.run()
ImageGenerator.save_recurrence_plot(result.recurrence_matrix_reverse,
                                    'recurrence_plot.png')

In [31]:
list(data.groupby(['session_id'])['high_performance'].max())

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1]

In [36]:
import numpy as np

Xtrain = pd.DataFrame(x)
Xtrain['high_performance'] = list(data.groupby(['session_id'])['high_performance'].max())
Xtrain['female_majority'] = list(data.groupby(['session_id'])['female_maj'].max())
#Xtrain = Xtrain.transpose()
Xtrain.columns = ["determinism", "recurrence_rate", "entropy", "longest_diagonal_line", "laminarity",
                 "divergence", "num_messages", "high_performance", 'female_majority']

In [38]:
Xtrain.columns[:7]

Index(['determinism', 'recurrence_rate', 'entropy', 'longest_diagonal_line',
       'laminarity', 'divergence', 'num_messages'],
      dtype='object')

In [87]:
formula_1 = "high_performance ~ "

for i in Xtrain.columns[:7]:
    formula_1 += i
    formula_1 += ' + '
    formula_1 += i + ':female_majority'
    formula_1 += ' + '
formula_1

'high_performance ~ determinism + determinism:female_majority + recurrence_rate + recurrence_rate:female_majority + entropy + entropy:female_majority + longest_diagonal_line + longest_diagonal_line:female_majority + laminarity + laminarity:female_majority + divergence + divergence:female_majority + num_messages + num_messages:female_majority + '

In [88]:
formula_1 = formula_1[:342]

In [101]:
Xtrain.astype(float).to_stata('x.dta')

In [90]:
formula_1

'high_performance ~ determinism + determinism:female_majority + recurrence_rate + recurrence_rate:female_majority + entropy + entropy:female_majority + longest_diagonal_line + longest_diagonal_line:female_majority + laminarity + laminarity:female_majority + divergence + divergence:female_majority + num_messages + num_messages:female_majority'

In [100]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

res1 = smf.logit(formula=formula_1, data=Xtrain.astype(float)).fit()

res1.summary()

         Current function value: 0.441243
         Iterations: 35


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       high_performance   No. Observations:                  129
Model:                          Logit   Df Residuals:                      114
Method:                           MLE   Df Model:                           14
Date:                Tue, 06 Jul 2021   Pseudo R-squ.:                  0.2452
Time:                        19:36:06   Log-Likelihood:                -56.920
converged:                      False   LL-Null:                       -75.409
Covariance Type:            nonrobust   LLR p-value:                 0.0007437
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                -0.0550      1.837     -0.030      0.976      -3.655       3.545
determinism                             -11.0250      7.962     -1.385      0.166     -26.630       4.580
determinism:female_majority              -9.1282     12.292     -0.743      0.458     -33.220      14.963
recurrence_rate                          30.7087     14.028      2.189      0.029       3.215      58.202
recurrence_rate:female_majority           2.2630     18.335      0.123      0.902     -33.672      38.198
entropy                                  -2.0910      2.502     -0.836      0.403      -6.995       2.813
entropy:female_majority                   6.2208      3.638      1.710      0.087      -0.909      13.351
longest_diagonal_line                 -4.647e-05   2.61e+05  -1.78e-10      1.000   -5.12e+05    5.12e+05
longest_diagonal_line:female_majority    -0.0142        nan        nan        nan         nan         nan
laminarity                               -0.9034      3.502     -0.258      0.796      -7.766       5.959
laminarity:female_majority                4.2083      6.422      0.655      0.512      -8.379      16.795
divergence                              -40.7107     32.166     -1.266      0.206    -103.756      22.334
divergence:female_majority             -106.1331     59.395     -1.787      0.074    -222.546      10.280
num_messages                             -0.0007   2.61e+05  -2.62e-09      1.000   -5.12e+05    5.12e+05
num_messages:female_majority             -0.0144        nan        nan        nan         nan         nan
=========================================================================================================
"""

In [68]:
Xtrain

,determinism,recurrence_rate,entropy,longest_diagonal_line,laminarity,divergence,num_messages,high_performance,female_majority
0,0.506158,0.266930,0.769781,78,0.641010,0.012821,78,0,1
1,0.400184,0.215632,0.551014,71,0.459982,0.014085,71,0,1
2,0.511312,0.202938,0.639641,33,0.524887,0.030303,33,0,0
3,0.376871,0.208333,0.719778,84,0.409524,0.011905,84,0,0
4,0.413636,0.207940,0.654058,46,0.404545,0.021739,46,0,1
...,...,...,...,...,...,...,...,...,...
124,0.460993,0.167658,0.752975,29,0.099291,0.034483,29,0,1
125,0.577778,0.200000,0.560819,30,0.405556,0.033333,30,0,1
126,0.460606,0.285467,0.840286,68,0.584848,0.014706,68,1,1
127,0.546502,0.304891,0.868985,124,0.700939,0.008065,124,1,0


In [13]:
Xtrain['num_messages']

0       78
1       71
2       33
3       84
4       46
      ... 
124     29
125     30
126     68
127    124
128     61
Name: num_messages, Length: 129, dtype: int64

In [16]:
# importing libraries
import statsmodels.api as sm

# building the model and fitting the data
log_reg = sm.Logit(np.asarray(Ytrain).astype(float), Xtrain.astype(float)).fit()


Optimization terminated successfully.
         Current function value: 0.496652
         Iterations 10


In [17]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  129
Model:                          Logit   Df Residuals:                      122
Method:                           MLE   Df Model:                            6
Date:                Tue, 06 Jul 2021   Pseudo R-squ.:                  0.1504
Time:                        18:39:45   Log-Likelihood:                -64.068
converged:                       True   LL-Null:                       -75.409
Covariance Type:            nonrobust   LLR p-value:                 0.0009104
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
determinism             -13.1874      5.180     -2.546      0.011     -23.341      -3.034
recurrence_rate          27.7146      8.057      3.440      0.001      11.924      43.505
entropy                   0.0911      1.576      0.058      0.954      -2.997       3.179
longest_diagonal_line    -0.0037        nan        nan        nan         nan         nan
laminarity                0.3905      2.542      0.154      0.878      -4.592       5.373
divergence              -61.4570     23.064     -2.665      0.008    -106.661     -16.253
num_messages             -0.0037        nan        nan        nan         nan         nan
=========================================================================================
"""

In [18]:
Xtrain = pd.DataFrame(x)
#Xtrain = Xtrain.transpose()
Xtrain.columns = ["determinism", "recurrence_rate", "entropy", "longest_diagonal_line", "laminarity",
                 "divergence", "num_messages"]

Ytrain = data.groupby(['session_id'])['female_maj'].unique()

In [19]:
# importing libraries
import statsmodels.api as sm

# building the model and fitting the data
log_reg = sm.Logit(np.asarray(Ytrain).astype(float), Xtrain.astype(float)).fit()


Optimization terminated successfully.
         Current function value: 0.649661
         Iterations 5


In [20]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  129
Model:                          Logit   Df Residuals:                      122
Method:                           MLE   Df Model:                            6
Date:                Tue, 06 Jul 2021   Pseudo R-squ.:                 0.01575
Time:                        18:40:05   Log-Likelihood:                -83.806
converged:                       True   LL-Null:                       -85.148
Covariance Type:            nonrobust   LLR p-value:                    0.8475
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
determinism               0.9225      3.028      0.305      0.761      -5.012       6.857
recurrence_rate          -1.9099      5.494     -0.348      0.728     -12.677       8.857
entropy                   1.3161      1.229      1.071      0.284      -1.093       3.725
longest_diagonal_line    -0.0053   1.85e+05   -2.9e-08      1.000   -3.62e+05    3.62e+05
laminarity                0.3955      1.632      0.242      0.809      -2.803       3.594
divergence               -4.6616     12.731     -0.366      0.714     -29.615      20.292
num_messages             -0.0053   1.85e+05  -2.87e-08      1.000   -3.62e+05    3.62e+05
=========================================================================================
"""

In [42]:
Xtrain

,determinism,recurrence_rate,entropy,longest_diagonal_line,laminarity,divergence,num_messages,female_maj,performance
0,0.428571,0.201646,0.752975,27,0.394558,0.037037,27,NaN,NaN
1,0.600000,0.300000,0.500402,10,0.000000,0.100000,10,NaN,NaN
2,0.653179,0.276800,0.929618,25,0.693642,0.040000,25,NaN,NaN
3,0.510417,0.198347,0.416431,44,0.395833,0.022727,44,NaN,NaN
4,0.426434,0.198025,0.709256,45,0.309227,0.022222,45,NaN,NaN
...,...,...,...,...,...,...,...,...,...
816,0.376871,0.208333,0.719778,84,0.409524,0.011905,84,NaN,NaN
817,0.462185,0.269841,0.848686,21,0.705882,0.047619,21,NaN,NaN
818,0.497951,0.238281,0.597450,64,0.674180,0.015625,64,NaN,NaN
819,0.692308,0.260000,0.500402,10,0.461538,0.100000,10,NaN,NaN


In [44]:
data.groupby(['team_id'])['female_maj'].max()

team_id
163793_163929_189654_163202         1
164313_196441_169447_176980         0
164987_168746_183931_164559         1
165634_166063_180976_201648         1
165974_166149_166141_185120         0
                                   ..
201620_174788_194314_175043         0
201973_172169_198974_185491         1
202919_Ratlife2001_198808_163573    0
203404_198251_191802_166085         1
Isabella_195466_187876_180275       1
Name: female_maj, Length: 821, dtype: int64

In [54]:
codes_list[:18]

team_id
163793_163929_189654_163202    [SMC, SSI, SMC, CRF, SSI, SSI, CRF, CE, SESU, ...
164313_196441_169447_176980    [SMC, SN, SMC, SESU, SN, SMC, SSI, SMC, SN, SESU]
164987_168746_183931_164559    [SMC, SMC, SMC, SMC, SMC, SESU, SSI, CM, SESU,...
165634_166063_180976_201648    [SMC, SMC, SMC, SESU, SN, SN, SN, CRF, CM, CP,...
165974_166149_166141_185120    [SMC, SESU, SSI, SSI, CRF, SESU, SESU, CRF, SE...
166559_188562_187413_185474    [SMC, SESU, SSI, SESU, CRF, SSI, SSI, CE, SSI,...
166720_163354_166483_190742    [SMC, SSI, SESU, CRF, CP, SSI, CRF, SSI, SSI, ...
166807_166145_190974_169429    [SSI, SSI, SSI, SSI, CE, CE, CRF, CRF, SN, SN,...
166874_184968_166265_189740    [SMC, SESU, SSI, SN, SESU, SSI, SSI, SSI, CRF,...
166943_182927_168426_174245    [SMC, SN, SSI, CRF, SSI, SN, SN, SSI, SSI, SES...
166968_201210_166778_196519    [SMC, SN, SSI, SMC, CM, CRF, SMC, CRF, SMC, SN...
167001_201680_178633_181167    [SMC, SMC, SMC, SMC, SMC, SMC, SMC, SMC, SMC, ...
167156_190133_167333